In [47]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset

In [48]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\notebooks\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [49]:
df = pd.read_excel("references\Subject_info_balanced.xlsx")

In [50]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class VGG3D(nn.Module):
    def __init__(self):
        super(VGG3D, self).__init__()
        # Define 3D convolution blocks with BatchNorm and ReLU activation
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm3d(32),
            nn.ReLU(inplace=True)
        )
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2, padding=(1, 0, 1))
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True)
        )
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2, padding=(1, 1, 0))

        self.conv3 = nn.Sequential(
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True),
            nn.Conv3d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm3d(128),
            nn.ReLU(inplace=True)
        )
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2, padding=(1, 0, 1))

        self.conv4 = nn.Sequential(
            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True)
        )
        self.pool4 = nn.MaxPool3d(kernel_size=2, stride=2, padding=(1, 1, 0))

        self.conv5 = nn.Sequential(
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm3d(256),
            nn.ReLU(inplace=True)
        )
        self.pool5 = nn.MaxPool3d(kernel_size=2, stride=2)

        # Pre-compute the flat features dimension after all pooling layers
        self.flat_features = self._calculate_flat_features(1, 193, 229, 193)

        # Define fully connected layers with BatchNorm, ReLU, and Dropout
        self.fc6 = nn.Sequential(
            nn.Linear(self.flat_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )
        
        self.fc7 = nn.Sequential(
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5)
        )

        self.fc8 = nn.Linear(512, 3)

    def _calculate_flat_features(self, channels, depth, height, width):
        x = torch.zeros((1, channels, depth, height, width))
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = self.pool3(self.conv3(x))
        x = self.pool4(self.conv4(x))
        x = self.pool5(self.conv5(x))
        return int(x.numel() // x.size(0))

    def forward(self, x):
        x = self.pool1(self.conv1(x))
        x = self.pool2(self.conv2(x))
        x = self.pool3(self.conv3(x))
        x = self.pool4(self.conv4(x))
        x = self.pool5(self.conv5(x))
        
        x = x.view(x.size(0), -1)
        
        x = self.fc6(x)
        x = self.fc7(x)
        x = self.fc8(x)
        x = F.softmax(x, dim=1)
        return x



In [52]:
from openpyxl import load_workbook
import pandas as pd
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim


def run_experiment(df, config):
    """Run the experiment with the given configuration on the preprocessed DataFrame."""
    train_dataset, val_dataset, test_dataset = load_datasets(df, config['image_type'], sample_size=config['sample_size'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    # Initialize model and training components
    device = torch.device("cuda")
    model = VGG3D().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    # Training and validation
    train_accuracies, val_accuracies, val_losses = train_and_validate(model, train_loader, val_loader, criterion, optimizer, label_mapping, config['num_epochs'], config['patience'], device)
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device)
    
    
    
    
  # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')

    # Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            # Reindex the existing DataFrame to ensure all columns are aligned and add new columns if necessary
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    return filename, train_accuracies[-1], val_accuracies[-1], test_accuracy

# Example configuration and use case
config = {
    'optimizer': 'Adam',
    'loss_criterion': 'BCEWithLogitsLoss',
    'num_epochs': 10,
    'batch_size': 2,
    'patience': 3,
    'Description' : 'VG33D advanced',
    'sample_size': 5
}

In [53]:
image_types = ['MRI_PET', 'MRI', 'PET']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)


Working on image type: MRI_PET


Epoch 1/10 - Train:   0%|          | 0/3 [00:15<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.04 GiB. GPU 